In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
import re
import spacy
from nltk.corpus import stopwords

In [4]:
train = pd.read_csv('./data/train.csv') 


In [5]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
train.index = train['id']
x_train = train['comment_text']
y_train = train.iloc[:,2:]

In [7]:
# if the entry is clean then the sum of the values of 
#toxic, severe_toxic, etc will be >=1 so the subtraction will
#be <1

y_train['clean'] = 1 - y_train.sum(axis=1) >= 1
y_train['clean'].astype('int64')

id
0000997932d777bf    1
000103f0d9cfb60f    1
000113f07ec002fd    1
0001b41b1c6bb37e    1
0001d958c54c6e35    1
                   ..
ffe987279560d7ff    1
ffea4adeee384e90    1
ffee36eab5c267c9    1
fff125370e4aaaf3    1
fff46fc426af1f9a    1
Name: clean, Length: 159571, dtype: int64

In [8]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'tagger', 'ner'])
stops = stopwords.words('english')

In [9]:
def normalize(comment, lowercase, remove_stops):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stops or (remove_stops and lemma not in stops):
                lemmatized.append(lemma)
    return ' '.join(lemmatized)

In [10]:
x_train_lemmatized = x_train.apply( 
    normalize, 
    lowercase=True, 
    remove_stops=True
)

/opt/miniconda3/envs/mlcysec/lib/python3.9/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


In [55]:
from collections import Counter
word_counts = dict()

for kind in y_train.columns:
    word_counts[kind] = Counter()
    comments = x_train_lemmatized[y_train[kind]==1]
    for _, comment in comments.iteritems():
        word_counts[kind].update(comment.split(" "))



In [24]:
import json
out_file = open('word_counts.json', 'w+')
json.dump(word_counts, out_file)

In [47]:
word, count = zip(*word_counts['toxic'].most_common(400)[::-1])

In [42]:
def most_common_words(word_class, num_words=50):
    words, counts = zip(*word_counts[word_class].most_common(num_words)[::-1])
    return words

In [2]:
temp = []
for kind in y_train.columns:
    a = list(most_common_words(kind))
    temp.append(a)

NameError: name 'y_train' is not defined

In [1]:
unique_words = list(set(temp))

NameError: name 'temp' is not defined